# <center>________________________________________________________________</center>

# <center>LANDING PREDICTION FOR THE SPACEX FALCON 9 ROCKET</center>

# <center>Part 4 - Maps of Launch Sites</center>

# <center>________________________________________________________________</center>

# Introduction
***

In this project, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of \\$62 million; other providers cost upward of \\$165 million each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous parts, we have visualized the SpaceX launch dataset and discovered some preliminary correlations between the launch site and success rates. In this part, we will be performing more interactive visual analytics on the locations of the launch sites.

# Libraries
***

In [ ]:
#!pip install folium
#!pip install wget
#!pip install pandas
#!pip install math

In [1]:
import folium
import wget
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

from math import sin, cos, sqrt, atan2, radians

# Data Acquisation
***

First, we will read the SpaceX dataset that we wrangled into a dataframe and print its summary:

In [2]:
df=pd.read_csv("https://github.com/efeyemez/Portfolio/raw/main/Datasets/SpaceX_Falcon_9/falcon9_wrangled.csv")
print(df.shape)
df.head(5)

(167, 18)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,8184.0,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


# Printing the Launch Sites
***

We will add each site's location on a map using site's latitude and longitude coordinates.

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `Class`
df = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]

# the coordinates for each site
launch_sites_df = df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. We will visualize those locations by pinning them on a map:

Now, we will add a circle for each launch site in data frame `launch_sites_df`:

In [4]:
start_coordinate = [30, -100]

# Initialize the map
site_map = folium.Map(location=start_coordinate, zoom_start=4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values.
# In addition, add Launch site name as a popup label

# CIRCLE

launch_sites_circle = folium.map.FeatureGroup()

for lat, lng, site in zip(launch_sites_df["Latitude"], launch_sites_df["Longitude"], launch_sites_df["LaunchSite"]):

    launch_sites_circle.add_child(
        folium.Circle([lat, lng], radius=50, color='#1E2F97', fill=True).add_child(folium.Popup(site))
    )


# MARKER

launch_sites_marker = folium.map.FeatureGroup()    
    
for lat, lng, site in zip(launch_sites_df["Latitude"], launch_sites_df["Longitude"], launch_sites_df["LaunchSite"]):

    launch_sites_marker.add_child(
        folium.map.Marker(
            [lat, lng],
            # Create an icon as a text label
            icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(-15,0),
            html='<div style="font-size: 12; color:#1E2F97;"><b>%s</b></div>' % site,
            )
        )
    )    
    

# Add to map
site_map.add_child(launch_sites_circle)
site_map.add_child(launch_sites_marker)

Now, we can explore the map by zoom-in/out the marked areas
, and try to answer some questions like:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

# Printing the Launches and the Outcomes
***

Next, we will enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. We will create markers for all launch records. If a launch was successful (Class=1), then we use a green marker and if a launch was failed, we use a red marker (Class=0):

Note that a launch only happens in one of the three launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

In [5]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

marker_color = []

for i in range(df.shape[0]):   
    
    if df.loc[i, "Class"] == 0:        
        marker_color.append("red")        
    else:        
        marker_color.append("green")

        
df["marker_color"] = marker_color
df["marker_color"].value_counts()

green    136
red       31
Name: marker_color, dtype: int64

In [6]:
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, row in df.iterrows():
    
    folium.map.Marker((row["Latitude"], row["Longitude"]), icon=folium.Icon(color='white',
                                                                  icon_color=row['marker_color'])).add_to(marker_cluster)
                           

site_map.add_child(marker_cluster)

From the color-labeled markers in marker clusters, we can easily identify which launch sites have relatively high success rates.

# Proximities and Distances
***

Next, we will explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as a railway). The coordinates of the mouse cursor will be displayed at the top right corner of the map:

In [7]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,)

site_map.add_child(mouse_position)
site_map

Now we can zoom in to a launch site and explore its proximity to see if we can find any railway, highway, coastline, etc. Then we can calculate the distance between two points on the map based on their Latitude and Longitude values using the following function:

In [8]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

## Closest Coastline

For launch site "KSC LC 39A", we will examine the closest coastline. We will gather the coordinates by using MousePosition, calculate the distance and print it on our map:

In [9]:
distance_coastline = calculate_distance(launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"], 28.61217, -80.59748)
distance_coastline

0.7803923118003601

In [10]:
# Create and add a folium.Marker on your selected closest coastline point on the map
distance_marker = folium.Marker(
    [28.61217, -80.59748],
    
    # Display the distance between coastline point and launch site using the icon property 
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#48494B;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )

site_map.add_child(distance_marker)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[[launch_sites_df.loc[1,"Latitude"], launch_sites_df.loc[1,"Longitude"]], [28.61217, -80.59748]],
                      weight=1)
site_map.add_child(lines)

When we zoom in to the launch site "KSC LC 39A", we can see the distance between closes coastline on our map. 

## Closest City, Railway, and Highway

Similarly, we can draw a line between the launch site to its closest city, railway, highway, etc. by using MousePosition to find their coordinates on the map.

A city map symbol may look like this:


<center>
    <img src="https://raw.githubusercontent.com/efeyemez/Portfolio/main/Datasets/SpaceX_Falcon_9/screenshots/city.png" />
</center>


A railway map symbol may look like this:


<center>
    <img src="https://raw.githubusercontent.com/efeyemez/Portfolio/main/Datasets/SpaceX_Falcon_9/screenshots/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://raw.githubusercontent.com/efeyemez/Portfolio/main/Datasets/SpaceX_Falcon_9/screenshots/highway.png" />
</center>


In [11]:
# City

distance_city = calculate_distance(launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"], 28.61225, -80.80771)
distance_city

distance_marker2 = folium.Marker(
    [28.61225, -80.80771],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#48494B;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )

site_map.add_child(distance_marker2)

lines2=folium.PolyLine(locations=[[launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"]], [28.61225, -80.80771]],
                      weight=1)
site_map.add_child(lines2)


# Railway
     
distance_railway = calculate_distance(launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"], 28.6104, -80.59691)
distance_railway

distance_marker3 = folium.Marker(
    [28.6104, -80.59691],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#48494B;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )

site_map.add_child(distance_marker3)

lines3=folium.PolyLine(locations=[[launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"]], [28.6104, -80.59691]],
                      weight=1)
site_map.add_child(lines3)


# Highway
     
distance_highway = calculate_distance(launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"], 28.61192, -80.59969)
distance_highway

distance_marker4 = folium.Marker(
    [28.61192, -80.59969],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#48494B;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )

site_map.add_child(distance_marker4)

lines3=folium.PolyLine(locations=[[launch_sites_df.loc[1, "Latitude"], launch_sites_df.loc[1, "Longitude"]], [28.61192, -80.59969]],
                      weight=1)
site_map.add_child(lines3)
     
     
     
     

After we plot distance lines to the proximities, we can answer some questions like:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

# <center>________________________________________________________________</center>